### IRIS AND BLINKING DETECTOR

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Webcam capture
cap = cv2.VideoCapture(0)

# Function to calculate the gaze ratio
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]

    # Horizontal gaze ratio
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to calculate blinking
def is_blinking(eye_landmarks, threshold=0.2):
    # Get vertical and horizontal landmarks for the eye
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    # Calculate vertical and horizontal distances
    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    # Blinking ratio
    blink_ratio = vertical_distance / horizontal_distance

    # Return True if blinking, False otherwise
    return blink_ratio < threshold

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Flip the frame horizontally for a mirror view
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame using Face Mesh
    results = face_mesh.process(frame_rgb)

    gaze_direction = "Unknown"
    blinking = False

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Get left and right iris centers and eye corners
            left_iris = face_landmarks.landmark[468]
            right_iris = face_landmarks.landmark[473]

            left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]  # Left eye corners
            right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]  # Right eye corners

            # Calculate gaze ratios for both eyes
            left_gaze_ratio = calculate_gaze_direction(left_iris, left_eye_corners)
            right_gaze_ratio = calculate_gaze_direction(right_iris, right_eye_corners)
            
            # Average gaze ratio
            avg_gaze_ratio = (left_gaze_ratio + right_gaze_ratio) / 2

            # Determine gaze direction
            if avg_gaze_ratio < 0.35:  # Adjust thresholds if needed
                gaze_direction = "Left"
            elif avg_gaze_ratio > 0.65:
                gaze_direction = "Right"
            else:
                gaze_direction = "Center"

            # Detect blinking for both eyes
            left_eye_landmarks = [
                face_landmarks.landmark[133],  # Left corner
                face_landmarks.landmark[159],  # Top
                face_landmarks.landmark[145],  # Bottom
                face_landmarks.landmark[33],  # Right corner
            ]
            right_eye_landmarks = [
                face_landmarks.landmark[362],  # Left corner
                face_landmarks.landmark[386],  # Top
                face_landmarks.landmark[374],  # Bottom
                face_landmarks.landmark[263],  # Right corner
            ]
            left_blinking = is_blinking(left_eye_landmarks)
            right_blinking = is_blinking(right_eye_landmarks)
            blinking = left_blinking or right_blinking

            # Visualize iris landmarks
            for landmark in [left_iris, right_iris]:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)

    # Display the gaze direction
    cv2.putText(frame, f"Gaze: {gaze_direction}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display blinking status
    blink_text = "Blinking" if blinking else "Not Blinking"
    cv2.putText(frame, f"{blink_text}", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the output frame
    cv2.imshow("MediaPipe Iris Tracking", frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### IRIS DETECTOR WITH UP AND DOWN EYE MOVEMENT

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Webcam capture
cap = cv2.VideoCapture(0)

# Function to calculate the horizontal gaze ratio
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]
    
    # Horizontal gaze ratio
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to calculate the vertical gaze ratio
def calculate_vertical_gaze_direction(iris_center, eye_corners):
    top_corner = eye_corners[1]
    bottom_corner = eye_corners[2]
    
    # Vertical gaze ratio
    vertical_ratio = (iris_center.y - top_corner.y) / (bottom_corner.y - top_corner.y)
    return vertical_ratio

# Function to calculate blinking
def is_blinking(eye_landmarks, threshold=0.2):
    # Get vertical and horizontal landmarks for the eye
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    # Calculate vertical and horizontal distances
    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    # Blinking ratio
    blink_ratio = vertical_distance / horizontal_distance

    # Return True if blinking, False otherwise
    return blink_ratio < threshold

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Flip the frame horizontally for a mirror view
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame using Face Mesh
    results = face_mesh.process(frame_rgb)

    gaze_direction = "Unknown"
    vertical_gaze_direction = "Unknown"
    blinking = False

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Get left and right iris centers and eye corners
            left_iris = face_landmarks.landmark[468]
            right_iris = face_landmarks.landmark[473]

            left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133], face_landmarks.landmark[145]]  # Left eye corners
            right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[386], face_landmarks.landmark[374]]  # Right eye corners

            # Calculate gaze ratios for both eyes (horizontal)
            left_gaze_ratio = calculate_gaze_direction(left_iris, left_eye_corners)
            right_gaze_ratio = calculate_gaze_direction(right_iris, right_eye_corners)

            # Average horizontal gaze ratio
            avg_gaze_ratio = (left_gaze_ratio + right_gaze_ratio) / 2

            # Determine horizontal gaze direction (Left, Center, Right)
            if avg_gaze_ratio < 0.35:
                gaze_direction = "Left"
            elif avg_gaze_ratio > 0.65:
                gaze_direction = "Right"
            else:
                gaze_direction = "Center"

            # Calculate vertical gaze ratio for both eyes (up/down)
            left_vertical_gaze_ratio = calculate_vertical_gaze_direction(left_iris, left_eye_corners)
            right_vertical_gaze_ratio = calculate_vertical_gaze_direction(right_iris, right_eye_corners)

            # Average vertical gaze ratio
            avg_vertical_gaze_ratio = (left_vertical_gaze_ratio + right_vertical_gaze_ratio) / 2

            # Determine vertical gaze direction (Up, Center, Down)
            if avg_vertical_gaze_ratio < 0.35:
                vertical_gaze_direction = "Up"
            elif avg_vertical_gaze_ratio > 0.65:
                vertical_gaze_direction = "Down"
            else:
                vertical_gaze_direction = "Center"

            # Detect blinking for both eyes
            left_eye_landmarks = [
                face_landmarks.landmark[133],  # Left corner
                face_landmarks.landmark[159],  # Top
                face_landmarks.landmark[145],  # Bottom
                face_landmarks.landmark[33],   # Right corner
            ]
            right_eye_landmarks = [
                face_landmarks.landmark[362],  # Left corner
                face_landmarks.landmark[386],  # Top
                face_landmarks.landmark[374],  # Bottom
                face_landmarks.landmark[263],  # Right corner
            ]
            left_blinking = is_blinking(left_eye_landmarks)
            right_blinking = is_blinking(right_eye_landmarks)
            blinking = left_blinking or right_blinking

            # Visualize iris landmarks
            for landmark in [left_iris, right_iris]:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)

    # Display the gaze direction
    cv2.putText(frame, f"Gaze Horizontal: {gaze_direction}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    
    # Display the vertical gaze direction
    cv2.putText(frame, f"Gaze Vertical: {vertical_gaze_direction}", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Display blinking status
    blink_text = "Blinking" if blinking else "Not Blinking"
    cv2.putText(frame, f"{blink_text}", (30, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the output frame
    cv2.imshow("MediaPipe Iris Tracking", frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### ONLY THE BASE IS WORKING

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time

# Initialize serial communication with Arduino
arduino = serial.Serial(port='COM3', baudrate=9600, timeout=.1)  # Replace 'COM3' with the appropriate port for your Arduino
time.sleep(2)  # Give the connection a moment to initialize

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Webcam capture
cap = cv2.VideoCapture(0)

# Function to calculate the gaze ratio
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]

    # Horizontal gaze ratio
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to calculate blinking
def is_blinking(eye_landmarks, threshold=0.2):
    # Get vertical and horizontal landmarks for the eye
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    # Calculate vertical and horizontal distances
    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    # Blinking ratio
    blink_ratio = vertical_distance / horizontal_distance

    # Return True if blinking, False otherwise
    return blink_ratio < threshold

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    # Flip the frame horizontally for a mirror view
    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame using Face Mesh
    results = face_mesh.process(frame_rgb)

    gaze_direction = "Unknown"
    blinking = False

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Get left and right iris centers and eye corners
            left_iris = face_landmarks.landmark[468]
            right_iris = face_landmarks.landmark[473]

            left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]  # Left eye corners
            right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]  # Right eye corners

            # Calculate gaze ratios for both eyes
            left_gaze_ratio = calculate_gaze_direction(left_iris, left_eye_corners)
            right_gaze_ratio = calculate_gaze_direction(right_iris, right_eye_corners)

            # Average gaze ratio
            avg_gaze_ratio = (left_gaze_ratio + right_gaze_ratio) / 2

            # Determine gaze direction
            if avg_gaze_ratio < 0.35:  # Adjust thresholds if needed
                gaze_direction = "Left"
                arduino.write(b'L\n')  # Send 'L' to Arduino for Left
            elif avg_gaze_ratio > 0.65:
                gaze_direction = "Right"
                arduino.write(b'R\n')  # Send 'R' to Arduino for Right
            else:
                gaze_direction = "Center"

            # Detect blinking for both eyes
            left_eye_landmarks = [
                face_landmarks.landmark[133],  # Left corner
                face_landmarks.landmark[159],  # Top
                face_landmarks.landmark[145],  # Bottom
                face_landmarks.landmark[33],  # Right corner
            ]
            right_eye_landmarks = [
                face_landmarks.landmark[362],  # Left corner
                face_landmarks.landmark[386],  # Top
                face_landmarks.landmark[374],  # Bottom
                face_landmarks.landmark[263],  # Right corner
            ]
            left_blinking = is_blinking(left_eye_landmarks)
            right_blinking = is_blinking(right_eye_landmarks)
            blinking = left_blinking or right_blinking

            if blinking:
                arduino.write(b'B\n')  # Send 'B' to Arduino for Blink

            # Visualize iris landmarks
            for landmark in [left_iris, right_iris]:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)

    # Display the gaze direction
    cv2.putText(frame, f"Gaze: {gaze_direction}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display blinking status
    blink_text = "Blinking" if blinking else "Not Blinking"
    cv2.putText(frame, f"{blink_text}", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the output frame
    cv2.imshow("MediaPipe Iris Tracking", frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### WORKING CLAW ONLY CODE


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time

# Initialize serial communication with Arduino
arduino = serial.Serial(port='COM3', baudrate=9600, timeout=.1)  # Adjust the port as needed
time.sleep(2)  # Allow time for the connection to initialize

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Webcam capture
cap = cv2.VideoCapture(0)

claw_state = False  # False: Closed, True: Open
blink_detected = False  # Tracks if blink was detected in the previous frame

# Function to calculate the gaze direction
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]

    # Horizontal gaze ratio
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to calculate blinking
def is_blinking(eye_landmarks, threshold=0.2):
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    blink_ratio = vertical_distance / horizontal_distance
    return blink_ratio < threshold

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    gaze_direction = "Unknown"
    current_blink = False

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_iris = face_landmarks.landmark[468]
            right_iris = face_landmarks.landmark[473]

            left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]
            right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]

            left_gaze_ratio = calculate_gaze_direction(left_iris, left_eye_corners)
            right_gaze_ratio = calculate_gaze_direction(right_iris, right_eye_corners)
            avg_gaze_ratio = (left_gaze_ratio + right_gaze_ratio) / 2

            if avg_gaze_ratio < 0.35:
                gaze_direction = "Left"
                arduino.write(b'L\n')  # Send 'L' to Arduino for left movement
            elif avg_gaze_ratio > 0.65:
                gaze_direction = "Right"
                arduino.write(b'R\n')  # Send 'R' to Arduino for right movement
            else:
                gaze_direction = "Center"
                arduino.write(b'C\n')  # Send 'C' to Arduino for center position

            left_eye_landmarks = [
                face_landmarks.landmark[133],  # Left corner
                face_landmarks.landmark[159],  # Top
                face_landmarks.landmark[145],  # Bottom
                face_landmarks.landmark[33],  # Right corner
            ]
            right_eye_landmarks = [
                face_landmarks.landmark[362],  # Left corner
                face_landmarks.landmark[386],  # Top
                face_landmarks.landmark[374],  # Bottom
                face_landmarks.landmark[263],  # Right corner
            ]
            left_blinking = is_blinking(left_eye_landmarks)
            right_blinking = is_blinking(right_eye_landmarks)
            current_blink = left_blinking or right_blinking

            # Handle blink state
            if current_blink and not blink_detected:
                blink_detected = True
                claw_state = not claw_state  # Toggle claw state
                # Send the servo position to Arduino based on claw state
                if claw_state:
                    arduino.write(b'OPEN\n')  # Open claw (110 degrees)
                else:
                    arduino.write(b'CLOSE\n')  # Close claw (90 degrees)
            elif not current_blink:
                blink_detected = False

            # Draw red dots for iris
            for landmark in [left_iris, right_iris]:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)

    cv2.putText(frame, f"Gaze: {gaze_direction}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Claw State: {'Open' if claw_state else 'Closed'}", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    cv2.imshow("MediaPipe Iris Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### WORKING CLAW AND BASE CODE

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time

# Initialize serial communication with Arduino
arduino = serial.Serial(port='COM3', baudrate=9600, timeout=.1)  # Adjust the port as needed
time.sleep(2)  # Allow time for the connection to initialize

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Webcam capture
cap = cv2.VideoCapture(0)

claw_state = False  # False: Closed, True: Open
blink_detected = False  # Tracks if blink was detected in the previous frame

# Function to calculate the gaze direction
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]

    # Horizontal gaze ratio
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to calculate blinking
def is_blinking(eye_landmarks, threshold=0.2):
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    blink_ratio = vertical_distance / horizontal_distance
    return blink_ratio < threshold

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    gaze_direction = "Unknown"
    current_blink = False

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            left_iris = face_landmarks.landmark[468]
            right_iris = face_landmarks.landmark[473]

            left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]
            right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]

            left_gaze_ratio = calculate_gaze_direction(left_iris, left_eye_corners)
            right_gaze_ratio = calculate_gaze_direction(right_iris, right_eye_corners)
            avg_gaze_ratio = (left_gaze_ratio + right_gaze_ratio) / 2

            # Adjust thresholds for gaze direction
            if avg_gaze_ratio < 0.35:
                gaze_direction = "LEFT"
                arduino.write(b'L\n')  # Send 'L' to Arduino for left movement
            elif avg_gaze_ratio > 0.65:
                gaze_direction = "RIGHT"
                arduino.write(b'R\n')  # Send 'R' to Arduino for right movement
            else:
                gaze_direction = "CENTER"
                arduino.write(b'C\n')  # Send 'C' to Arduino for center position

            left_eye_landmarks = [
                face_landmarks.landmark[133],  # Left corner
                face_landmarks.landmark[159],  # Top
                face_landmarks.landmark[145],  # Bottom
                face_landmarks.landmark[33],  # Right corner
            ]
            right_eye_landmarks = [
                face_landmarks.landmark[362],  # Left corner
                face_landmarks.landmark[386],  # Top
                face_landmarks.landmark[374],  # Bottom
                face_landmarks.landmark[263],  # Right corner
            ]
            left_blinking = is_blinking(left_eye_landmarks)
            right_blinking = is_blinking(right_eye_landmarks)
            current_blink = left_blinking or right_blinking

            # Handle blink state
            if current_blink and not blink_detected:
                blink_detected = True
                claw_state = not claw_state  # Toggle claw state
                # Send the servo position to Arduino based on claw state
                if claw_state:
                    arduino.write(b'OPEN\n')  # Open claw (110 degrees)
                else:
                    arduino.write(b'CLOSE\n')  # Close claw (90 degrees)
            elif not current_blink:
                blink_detected = False

            # Draw red dots for iris
            for landmark in [left_iris, right_iris]:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)

    # Display gaze direction on the frame
    cv2.putText(frame, f"Gaze: {gaze_direction}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Claw State: {'Open' if claw_state else 'Closed'}", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)

    # Show the output frame
    cv2.imshow("MediaPipe Iris Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


### EYEBROW MOVEMENT DETECTION

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Set up webcam
cap = cv2.VideoCapture(0)

# Define a function to calculate the eyebrow movement (detect upward movement only)
def calculate_eyebrow_movement(landmarks, left_brow_indices, right_brow_indices, baseline, movement_threshold):
    # Get the vertical position (y-coordinate) of the eyebrows
    left_brow_y = np.mean([landmarks[i][1] for i in left_brow_indices])
    right_brow_y = np.mean([landmarks[i][1] for i in right_brow_indices])

    # Calculate the average vertical position of both eyebrows
    avg_brow_y = (left_brow_y + right_brow_y) / 2

    # If no baseline, initialize it with the current position
    if baseline is None:
        baseline = avg_brow_y
        return "Neutral", baseline

    # Detect upward movement by comparing with the baseline
    if avg_brow_y < baseline - movement_threshold:
        return "Up", baseline
    else:
        # Update baseline slightly to adapt to natural movements
        baseline = 0.9 * baseline + 0.1 * avg_brow_y
        return "Neutral", baseline


# Correct indices for the left and right eyebrows (provided by you)
LEFT_BROW = [46, 53, 52, 65, 55, 63, 105, 66, 107]  # Left eyebrow indices
RIGHT_BROW = [276, 283, 282, 295, 285, 293, 334, 296, 336]  # Right eyebrow indices

# Initialize baseline and movement threshold
baseline_position = None
movement_threshold = 0.01  # Adjust this value to control sensitivity

# Initialize the Face Mesh model
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()

        if not ret:
            break

        # Convert the frame to RGB for MediaPipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Process the frame to get face landmarks
        results = face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                # Extract landmarks as a list of (x, y, z) tuples
                landmarks = [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark]

                # Check eyebrow movement
                eyebrow_state, baseline_position = calculate_eyebrow_movement(
                    landmarks, LEFT_BROW, RIGHT_BROW, baseline_position, movement_threshold
                )

                # Display the eyebrow state on the frame
                cv2.putText(frame, f"Eyebrow Movement: {eyebrow_state}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Draw only the eyebrow landmarks (left and right)
                for i in LEFT_BROW:
                    x, y, _ = landmarks[i]
                    cv2.circle(frame, (int(x * frame.shape[1]), int(y * frame.shape[0])), 2, (0, 255, 0), -1)
                for i in RIGHT_BROW:
                    x, y, _ = landmarks[i]
                    cv2.circle(frame, (int(x * frame.shape[1]), int(y * frame.shape[0])), 2, (0, 255, 0), -1)

        # Display the frame with the eyebrow landmarks and movement status
        cv2.imshow("Eyebrow Movement Detection", frame)

        # Exit the loop when the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Release the webcam and close the window
cap.release()
cv2.destroyAllWindows()


### EYEBROW WITH ARDUINO ELBOW

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import serial  # For Arduino communication
import time  # To allow time for serial communication setup

# Initialize MediaPipe Face Mesh
mp_face_mesh = mp.solutions.face_mesh

# Arduino Serial Port Configuration
arduino = serial.Serial(port="COM3", baudrate=9600, timeout=1)  # Update COM3 to your port
time.sleep(2)  # Wait for Arduino to initialize

# Define a function to calculate the eyebrow movement (detect upward movement only)
def calculate_eyebrow_movement(landmarks, left_brow_indices, right_brow_indices, baseline, movement_threshold):
    left_brow_y = np.mean([landmarks[i][1] for i in left_brow_indices])
    right_brow_y = np.mean([landmarks[i][1] for i in right_brow_indices])
    avg_brow_y = (left_brow_y + right_brow_y) / 2

    if baseline is None:
        baseline = avg_brow_y
        return "Neutral", baseline

    if avg_brow_y < baseline - movement_threshold:
        return "Up", baseline
    else:
        baseline = 0.9 * baseline + 0.1 * avg_brow_y
        return "Neutral", baseline


# Eyebrow landmark indices
LEFT_BROW = [46, 53, 52, 65, 55, 63, 105, 66, 107]
RIGHT_BROW = [276, 283, 282, 295, 285, 293, 334, 296, 336]

# Initialize baseline and movement threshold
baseline_position = None
movement_threshold = 0.01

# Initialize Face Mesh
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    cap = cv2.VideoCapture(0)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                landmarks = [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark]

                # Detect eyebrow movement
                eyebrow_state, baseline_position = calculate_eyebrow_movement(
                    landmarks, LEFT_BROW, RIGHT_BROW, baseline_position, movement_threshold
                )

                # Send commands to Arduino based on the eyebrow state
                if eyebrow_state == "Up":
                    arduino.write(b"UP\n")  # Send "UP" command
                elif eyebrow_state == "Neutral":
                    arduino.write(b"DOWN\n")  # Send "DOWN" command

                # Display the eyebrow state on the frame
                cv2.putText(frame, f"Eyebrow Movement: {eyebrow_state}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

                # Draw the eyebrow landmarks
                for i in LEFT_BROW:
                    x, y, _ = landmarks[i]
                    cv2.circle(frame, (int(x * frame.shape[1]), int(y * frame.shape[0])), 2, (0, 255, 0), -1)
                for i in RIGHT_BROW:
                    x, y, _ = landmarks[i]
                    cv2.circle(frame, (int(x * frame.shape[1]), int(y * frame.shape[0])), 2, (0, 255, 0), -1)

        cv2.imshow("Eyebrow Movement Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Clean up
cap.release()
cv2.destroyAllWindows()
arduino.close()


### MOUTH OPEN OR CLOSED DETECTION PYTHON ONLY CODE

In [ ]:
import cv2
import mediapipe as mp

# Initialize mediapipe face mesh and drawing utilities
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Start webcam feed
cap = cv2.VideoCapture(0)

# Initialize the face mesh model
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert the frame to RGB (MediaPipe needs RGB images)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)
        
        # Clear the frame to remove face mesh landmarks
        annotated_frame = frame.copy()

        # Process detected faces
        if results.multi_face_landmarks:
            for landmarks in results.multi_face_landmarks:
                # Get the upper and lower lip landmarks
                upper_lip_y = landmarks.landmark[61].y
                lower_lip_y = landmarks.landmark[17].y

                # Calculate the distance between the upper and lower lips (y-coordinate difference)
                lip_distance = abs(upper_lip_y - lower_lip_y)

                # Get landmarks for a reference distance to calculate dynamic threshold (between eyes or nose)
                left_eye = landmarks.landmark[33]
                right_eye = landmarks.landmark[263]
                # Calculate the width between eyes as a reference
                eye_distance = abs(left_eye.x - right_eye.x)

                # Dynamic threshold based on face distance (based on eye width)
                # The closer you are, the more sensitive the mouth open detection will be
                # The farther you are, the less sensitive it becomes
                threshold = 0.025 + (eye_distance * 0.1)  # Dynamic adjustment for threshold

                # Detect if the mouth is open or closed
                if lip_distance > threshold:  # Dynamic threshold for mouth open detection
                    mouth_status = "Mouth Open"
                else:
                    mouth_status = "Mouth Closed"

                # Display the mouth status on the frame
                cv2.putText(annotated_frame, mouth_status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # Display the annotated frame without face mesh
        cv2.imshow("Mouth Open/Closed Detection", annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam feed and close the window
    cap.release()
    cv2.destroyAllWindows()


### MOUTH OPEN WITH ARDUINO


In [ ]:
import cv2
import mediapipe as mp
import serial
import time

# Initialize serial communication (adjust 'COM3' and 9600 as needed for your setup)
arduino = serial.Serial('COM3', 9600)
time.sleep(2)  # Give the connection a second to initialize

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh

cap = cv2.VideoCapture(0)

with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(rgb_frame)
        annotated_frame = frame.copy()

        if results.multi_face_landmarks:
            for landmarks in results.multi_face_landmarks:
                # Get lip landmarks
                upper_lip_y = landmarks.landmark[61].y
                lower_lip_y = landmarks.landmark[17].y

                # Calculate distance and dynamic threshold
                lip_distance = abs(upper_lip_y - lower_lip_y)
                left_eye = landmarks.landmark[33]
                right_eye = landmarks.landmark[263]
                eye_distance = abs(left_eye.x - right_eye.x)
                threshold = 0.025 + (eye_distance * 0.1)

                # Determine mouth status
                if lip_distance > threshold:
                    mouth_status = "Mouth Open"
                    arduino.write(b'1')  # Send signal to Arduino
                else:
                    mouth_status = "Mouth Closed"
                    arduino.write(b'0')  # Send signal to Arduino

                # Display mouth status
                cv2.putText(annotated_frame, mouth_status, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        cv2.imshow("Mouth Open/Closed Detection", annotated_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()
arduino.close()


### COMPLETE PYTHON CODE CONTROL SIGNAL FOR 4DOF ROBOTIC ARM USING MOUTH, EYES AND EYEBROWS

In [35]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time


# Initialize serial communication with Arduino
arduino = serial.Serial('COM3', 9600)
time.sleep(2)  # Allow time for the connection to initialize

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Webcam capture
cap = cv2.VideoCapture(0)

claw_state = False  # False: Closed, True: Open
blink_detected = False  # Tracks if blink was detected in the previous frame

# Eyebrow landmark indices
LEFT_BROW = [46, 53, 52, 65, 55, 63, 105, 66, 107]
RIGHT_BROW = [276, 283, 282, 295, 285, 293, 334, 296, 336]

# Function to calculate the gaze direction
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to calculate blinking
def is_blinking(eye_landmarks, threshold=0.2):
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    blink_ratio = vertical_distance / horizontal_distance
    return blink_ratio < threshold

# Function to calculate the eyebrow movement (detect upward movement only)
def calculate_eyebrow_movement(landmarks, left_brow_indices, right_brow_indices, baseline, movement_threshold):
    left_brow_y = np.mean([landmarks[i][1] for i in left_brow_indices])
    right_brow_y = np.mean([landmarks[i][1] for i in right_brow_indices])
    avg_brow_y = (left_brow_y + right_brow_y) / 2

    if baseline is None:
        baseline = avg_brow_y
        return "Neutral", baseline

    if avg_brow_y < baseline - movement_threshold:
        return "Up", baseline
    else:
        baseline = 0.9 * baseline + 0.1 * avg_brow_y
        return "Neutral", baseline

# Initialize baseline and movement threshold for eyebrow detection
baseline_position = None
movement_threshold = 0.01

# Function to detect mouth open/closed status
def detect_mouth_status(landmarks):
    upper_lip_y = landmarks.landmark[61].y
    lower_lip_y = landmarks.landmark[17].y
    lip_distance = abs(upper_lip_y - lower_lip_y)
    
    left_eye = landmarks.landmark[33]
    right_eye = landmarks.landmark[263]
    eye_distance = abs(left_eye.x - right_eye.x)
    
    threshold = 0.025 + (eye_distance * 0.1)
    
    if lip_distance > threshold:
        return "Mouth Open"
    else:
        return "Mouth Closed"

# Function to process frames
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    gaze_direction = "Unknown"
    current_blink = False
    mouth_status = "Mouth Closed"

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Iris and gaze detection
            left_iris = face_landmarks.landmark[468]
            right_iris = face_landmarks.landmark[473]

            left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]
            right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]

            left_gaze_ratio = calculate_gaze_direction(left_iris, left_eye_corners)
            right_gaze_ratio = calculate_gaze_direction(right_iris, right_eye_corners)
            avg_gaze_ratio = (left_gaze_ratio + right_gaze_ratio) / 2

            # Adjust thresholds for gaze direction
            if avg_gaze_ratio < 0.35:
                gaze_direction = "LEFT"
                arduino.write(b'L\n')  # Send 'L' to Arduino for left movement
            elif avg_gaze_ratio > 0.65:
                gaze_direction = "RIGHT"
                arduino.write(b'R\n')  # Send 'R' to Arduino for right movement
            else:
                gaze_direction = "CENTER"
                arduino.write(b'C\n')  # Send 'C' to Arduino for center position

            # Eye blink detection
            left_eye_landmarks = [
                face_landmarks.landmark[133],  # Left corner
                face_landmarks.landmark[159],  # Top
                face_landmarks.landmark[145],  # Bottom
                face_landmarks.landmark[33],   # Right corner
            ]
            right_eye_landmarks = [
                face_landmarks.landmark[362],  # Left corner
                face_landmarks.landmark[386],  # Top
                face_landmarks.landmark[374],  # Bottom
                face_landmarks.landmark[263],  # Right corner
            ]
            left_blinking = is_blinking(left_eye_landmarks)
            right_blinking = is_blinking(right_eye_landmarks)
            current_blink = left_blinking or right_blinking

            # Handle blink state for claw control
            if current_blink and not blink_detected:
                blink_detected = True
                claw_state = not claw_state  # Toggle claw state
                if claw_state:
                    arduino.write(b'OPEN\n')  # Open claw
                else:
                    arduino.write(b'CLOSE\n')  # Close claw
            elif not current_blink:
                blink_detected = False

            # Detect eyebrow movement
            eyebrow_state, baseline_position = calculate_eyebrow_movement(
                [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark],
                LEFT_BROW, RIGHT_BROW, baseline_position, movement_threshold
            )

            # Send commands to Arduino based on eyebrow state
            if eyebrow_state == "Up":
                arduino.write(b"UP\n")  # Send "UP" command
            elif eyebrow_state == "Neutral":
                arduino.write(b"DOWN\n")  # Send "DOWN" command

            # Detect mouth status
            mouth_status = detect_mouth_status(face_landmarks)

            # Send mouth status to Arduino
            if mouth_status == "Mouth Open":
                arduino.write(b'EXTEND\n')  # Send signal to Arduino
            else:
                arduino.write(b'RETRACT\n')  # Send signal to Arduino

            # Draw red dots for iris
            for landmark in [left_iris, right_iris]:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)

            # Display the gaze direction, claw state, eyebrow, and mouth status on the frame
            cv2.putText(frame, f"Gaze: {gaze_direction}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Claw State: {'Open' if claw_state else 'Closed'}", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.putText(frame, f"Eyebrow Movement: {eyebrow_state}", (50, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Mouth Status: {mouth_status}", (50, 170), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Show the output frame
    cv2.imshow("Face Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()
arduino.close()


### COMPLETE FACIAL GESTURE DETECTION (NO ARDUINO)

In [ ]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Webcam capture
cap = cv2.VideoCapture(0)

claw_state = False  # False: Closed, True: Open
blink_detected = False  # Tracks if blink was detected in the previous frame

# Eyebrow landmark indices
LEFT_BROW = [46, 53, 52, 65, 55, 63, 105, 66, 107]
RIGHT_BROW = [276, 283, 282, 295, 285, 293, 334, 296, 336]

# Function to calculate the gaze direction
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to calculate blinking
def is_blinking(eye_landmarks, threshold=0.2):
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    blink_ratio = vertical_distance / horizontal_distance
    return blink_ratio < threshold

# Function to calculate the eyebrow movement (detect upward movement only)
def calculate_eyebrow_movement(landmarks, left_brow_indices, right_brow_indices, baseline, movement_threshold):
    left_brow_y = np.mean([landmarks[i][1] for i in left_brow_indices])
    right_brow_y = np.mean([landmarks[i][1] for i in right_brow_indices])
    avg_brow_y = (left_brow_y + right_brow_y) / 2

    if baseline is None:
        baseline = avg_brow_y
        return "Neutral", baseline

    if avg_brow_y < baseline - movement_threshold:
        return "Up", baseline
    else:
        baseline = 0.9 * baseline + 0.1 * avg_brow_y
        return "Neutral", baseline

# Initialize baseline and movement threshold for eyebrow detection
baseline_position = None
movement_threshold = 0.01

# Function to detect mouth open/closed status
def detect_mouth_status(landmarks):
    upper_lip_y = landmarks.landmark[61].y
    lower_lip_y = landmarks.landmark[17].y
    lip_distance = abs(upper_lip_y - lower_lip_y)
    
    left_eye = landmarks.landmark[33]
    right_eye = landmarks.landmark[263]
    eye_distance = abs(left_eye.x - right_eye.x)
    
    threshold = 0.025 + (eye_distance * 0.1)
    
    if lip_distance > threshold:
        return "Mouth Open"
    else:
        return "Mouth Closed"

# Process webcam frames
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        print("Ignoring empty camera frame.")
        continue

    frame = cv2.flip(frame, 1)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    gaze_direction = "Unknown"
    current_blink = False
    mouth_status = "Mouth Closed"

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Iris and gaze detection
            left_iris = face_landmarks.landmark[468]
            right_iris = face_landmarks.landmark[473]

            left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]
            right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]

            left_gaze_ratio = calculate_gaze_direction(left_iris, left_eye_corners)
            right_gaze_ratio = calculate_gaze_direction(right_iris, right_eye_corners)
            avg_gaze_ratio = (left_gaze_ratio + right_gaze_ratio) / 2

            if avg_gaze_ratio < 0.35:
                gaze_direction = "LEFT"
            elif avg_gaze_ratio > 0.65:
                gaze_direction = "RIGHT"
            else:
                gaze_direction = "CENTER"

            # Eye blink detection
            left_eye_landmarks = [
                face_landmarks.landmark[133],  # Left corner
                face_landmarks.landmark[159],  # Top
                face_landmarks.landmark[145],  # Bottom
                face_landmarks.landmark[33],   # Right corner
            ]
            right_eye_landmarks = [
                face_landmarks.landmark[362],  # Left corner
                face_landmarks.landmark[386],  # Top
                face_landmarks.landmark[374],  # Bottom
                face_landmarks.landmark[263],  # Right corner
            ]
            left_blinking = is_blinking(left_eye_landmarks)
            right_blinking = is_blinking(right_eye_landmarks)
            current_blink = left_blinking or right_blinking

            # Detect eyebrow movement
            eyebrow_state, baseline_position = calculate_eyebrow_movement(
                [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark],
                LEFT_BROW, RIGHT_BROW, baseline_position, movement_threshold
            )

            # Detect mouth status
            mouth_status = detect_mouth_status(face_landmarks)

            # Draw red dots for iris
            for landmark in [left_iris, right_iris]:
                x = int(landmark.x * frame.shape[1])
                y = int(landmark.y * frame.shape[0])
                cv2.circle(frame, (x, y), 3, (0, 0, 255), -1)

            # Display the gaze direction, blink status, eyebrow, and mouth status on the frame
            cv2.putText(frame, f"Gaze: {gaze_direction}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Blink: {'Detected' if current_blink else 'Not Detected'}", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
            cv2.putText(frame, f"Eyebrow Movement: {eyebrow_state}", (50, 130), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f"Mouth Status: {mouth_status}", (50, 170), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

    # Show the output frame
    cv2.imshow("Face Tracking", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap.release()
cv2.destroyAllWindows()


### COMPLETE CODE WITH 2 VIDEO FEED

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time

# Initialize serial communication with Arduino
arduino = serial.Serial('COM3', 9600)
time.sleep(2)  # Allow time for the connection to initialize

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Webcam capture for face control
cap1 = cv2.VideoCapture(0)

# Webcam capture for second camera (bottom video)
cap2 = cv2.VideoCapture(1)

claw_state = False  # False: Closed, True: Open
blink_detected = False  # Tracks if blink was detected in the previous frame

# Eyebrow landmark indices
LEFT_BROW = [46, 53, 52, 65, 55, 63, 105, 66, 107]
RIGHT_BROW = [276, 283, 282, 295, 285, 293, 334, 296, 336]

# Function to calculate the gaze direction
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to calculate blinking
def is_blinking(eye_landmarks, threshold=0.2):
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    blink_ratio = vertical_distance / horizontal_distance
    return blink_ratio < threshold

# Function to calculate the eyebrow movement (detect upward movement only)
def calculate_eyebrow_movement(landmarks, left_brow_indices, right_brow_indices, baseline, movement_threshold):
    left_brow_y = np.mean([landmarks[i][1] for i in left_brow_indices])
    right_brow_y = np.mean([landmarks[i][1] for i in right_brow_indices])
    avg_brow_y = (left_brow_y + right_brow_y) / 2

    if baseline is None:
        baseline = avg_brow_y
        return "Neutral", baseline

    if avg_brow_y < baseline - movement_threshold:
        return "Up", baseline
    else:
        baseline = 0.9 * baseline + 0.1 * avg_brow_y
        return "Neutral", baseline

# Initialize baseline and movement threshold for eyebrow detection
baseline_position = None
movement_threshold = 0.01

# Function to detect mouth open/closed status
def detect_mouth_status(landmarks):
    upper_lip_y = landmarks.landmark[61].y
    lower_lip_y = landmarks.landmark[17].y
    lip_distance = abs(upper_lip_y - lower_lip_y)
    
    left_eye = landmarks.landmark[33]
    right_eye = landmarks.landmark[263]
    eye_distance = abs(left_eye.x - right_eye.x)
    
    threshold = 0.025 + (eye_distance * 0.1)
    
    if lip_distance > threshold:
        return "Mouth Open"
    else:
        return "Mouth Closed"

# Function to process frames
while cap1.isOpened() and cap2.isOpened():
    success1, frame1 = cap1.read()
    success2, frame2 = cap2.read()
    
    if not success1 or not success2:
        print("Ignoring empty camera frame.")
        continue

    frame1 = cv2.flip(frame1, 1)
    frame2 = cv2.flip(frame2, 1)
    frame1_rgb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame1_rgb)

    gaze_direction = "Unknown"
    current_blink = False
    mouth_status = "Mouth Closed"

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Iris and gaze detection
            left_iris = face_landmarks.landmark[468]
            right_iris = face_landmarks.landmark[473]

            left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]
            right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]

            left_gaze_ratio = calculate_gaze_direction(left_iris, left_eye_corners)
            right_gaze_ratio = calculate_gaze_direction(right_iris, right_eye_corners)
            avg_gaze_ratio = (left_gaze_ratio + right_gaze_ratio) / 2

            # Adjust thresholds for gaze direction
            if avg_gaze_ratio < 0.35:
                gaze_direction = "LEFT"
                arduino.write(b'L\n')  # Send 'L' to Arduino for left movement
            elif avg_gaze_ratio > 0.65:
                gaze_direction = "RIGHT"
                arduino.write(b'R\n')  # Send 'R' to Arduino for right movement
            else:
                gaze_direction = "CENTER"
                arduino.write(b'C\n')  # Send 'C' to Arduino for center position

            # Eye blink detection
            left_eye_landmarks = [
                face_landmarks.landmark[133],  # Left corner
                face_landmarks.landmark[159],  # Top
                face_landmarks.landmark[145],  # Bottom
                face_landmarks.landmark[33],   # Right corner
            ]
            right_eye_landmarks = [
                face_landmarks.landmark[362],  # Left corner
                face_landmarks.landmark[386],  # Top
                face_landmarks.landmark[374],  # Bottom
                face_landmarks.landmark[263],  # Right corner
            ]
            left_blinking = is_blinking(left_eye_landmarks)
            right_blinking = is_blinking(right_eye_landmarks)
            current_blink = left_blinking or right_blinking

            # Handle blink state for claw control
            if current_blink and not blink_detected:
                blink_detected = True
                claw_state = not claw_state  # Toggle claw state
                if claw_state:
                    arduino.write(b'OPEN\n')  # Open claw
                else:
                    arduino.write(b'CLOSE\n')  # Close claw
            elif not current_blink:
                blink_detected = False

            # Detect eyebrow movement
            eyebrow_state, baseline_position = calculate_eyebrow_movement(
                [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark],
                LEFT_BROW, RIGHT_BROW, baseline_position, movement_threshold
            )

            # Send commands to Arduino based on eyebrow state
            if eyebrow_state == "Up":
                arduino.write(b"UP\n")  # Send "UP" command
            elif eyebrow_state == "Neutral":
                arduino.write(b"DOWN\n")  # Send "DOWN" command

            # Detect mouth status
            mouth_status = detect_mouth_status(face_landmarks)

            # Send mouth status to Arduino
            if mouth_status == "Mouth Open":
                arduino.write(b'EXTEND\n')  # Send signal to Arduino
            else:
                arduino.write(b'RETRACT\n')  # Send signal to Arduino

            # Draw the text information on the frame
            cv2.putText(frame1, f"Gaze: {gaze_direction}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)
            cv2.putText(frame1, f"Claw: {'Open' if claw_state else 'Closed'}", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1)
            cv2.putText(frame1, f"Eyebrow: {eyebrow_state}", (30, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)
            cv2.putText(frame1, f"Mouth: {mouth_status}", (30, 170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 1)

    # Stack the two frames vertically
    stacked_frames = np.vstack((frame1, frame2))

    # Show the output
    cv2.imshow("Face Tracking and Camera Feed", stacked_frames)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Clean up
cap1.release()
cap2.release()
cv2.destroyAllWindows()


### COMPLETE CODE WITH 2 VIDEO FEED UPGRADE

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time

# Initialize serial communication with Arduino
arduino = serial.Serial('COM3', 9600)
time.sleep(2)  # Allow time for the connection to initialize

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Webcam capture for face control
cap1 = cv2.VideoCapture(0)

# Webcam capture for second camera (bottom video)
cap2 = cv2.VideoCapture(1)

claw_state = False  # False: Closed, True: Open
blink_detected = False  # Tracks if blink was detected in the previous frame

# Eyebrow landmark indices
LEFT_BROW = [46, 53, 52, 65, 55, 63, 105, 66, 107]
RIGHT_BROW = [276, 283, 282, 295, 285, 293, 334, 296, 336]

# Function to calculate the gaze direction
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to calculate blinking
def is_blinking(eye_landmarks, threshold=0.2):
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    blink_ratio = vertical_distance / horizontal_distance
    return blink_ratio < threshold

# Function to calculate the eyebrow movement (detect upward movement only)
def calculate_eyebrow_movement(landmarks, left_brow_indices, right_brow_indices, baseline, movement_threshold):
    left_brow_y = np.mean([landmarks[i][1] for i in left_brow_indices])
    right_brow_y = np.mean([landmarks[i][1] for i in right_brow_indices])
    avg_brow_y = (left_brow_y + right_brow_y) / 2

    if baseline is None:
        baseline = avg_brow_y
        return "Neutral", baseline

    if avg_brow_y < baseline - movement_threshold:
        return "Up", baseline
    else:
        baseline = 0.9 * baseline + 0.1 * avg_brow_y
        return "Neutral", baseline

# Initialize baseline and movement threshold for eyebrow detection
baseline_position = None
movement_threshold = 0.01

# Function to detect mouth open/closed status
def detect_mouth_status(landmarks):
    upper_lip_y = landmarks.landmark[61].y
    lower_lip_y = landmarks.landmark[17].y
    lip_distance = abs(upper_lip_y - lower_lip_y)
    
    left_eye = landmarks.landmark[33]
    right_eye = landmarks.landmark[263]
    eye_distance = abs(left_eye.x - right_eye.x)
    
    threshold = 0.025 + (eye_distance * 0.1)
    
    if lip_distance > threshold:
        return "Mouth Open"
    else:
        return "Mouth Closed"

# Function to process frames
while cap1.isOpened() and cap2.isOpened():
    success1, frame1 = cap1.read()
    success2, frame2 = cap2.read()
    
    if not success1 or not success2:
        print("Ignoring empty camera frame.")
        continue

    frame1 = cv2.flip(frame1, 1)
    frame2 = cv2.flip(frame2, 1)
    
    # Resize the frames to change the video display size
    frame1 = cv2.resize(frame1, (640, 480))  # Smaller face control feed
    frame2 = cv2.resize(frame2, (640, 480))  # Resize second feed to match width
    
    frame1_rgb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame1_rgb)

    gaze_direction = "Unknown"
    current_blink = False
    mouth_status = "Mouth Closed"

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            # Iris and gaze detection
            left_iris = face_landmarks.landmark[468]
            right_iris = face_landmarks.landmark[473]

            left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]
            right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]

            left_gaze_ratio = calculate_gaze_direction(left_iris, left_eye_corners)
            right_gaze_ratio = calculate_gaze_direction(right_iris, right_eye_corners)
            avg_gaze_ratio = (left_gaze_ratio + right_gaze_ratio) / 2

            # Adjust thresholds for gaze direction
            if avg_gaze_ratio < 0.35:
                gaze_direction = "LEFT"
                arduino.write(b'L\n')  # Send 'L' to Arduino for left movement
            elif avg_gaze_ratio > 0.65:
                gaze_direction = "RIGHT"
                arduino.write(b'R\n')  # Send 'R' to Arduino for right movement
            else:
                gaze_direction = "CENTER"
                arduino.write(b'C\n')  # Send 'C' to Arduino for center position

            # Eye blink detection
            left_eye_landmarks = [
                face_landmarks.landmark[133],  # Left corner
                face_landmarks.landmark[159],  # Top
                face_landmarks.landmark[145],  # Bottom
                face_landmarks.landmark[33],   # Right corner
            ]
            right_eye_landmarks = [
                face_landmarks.landmark[362],  # Left corner
                face_landmarks.landmark[386],  # Top
                face_landmarks.landmark[374],  # Bottom
                face_landmarks.landmark[263],  # Right corner
            ]
            left_blinking = is_blinking(left_eye_landmarks)
            right_blinking = is_blinking(right_eye_landmarks)
            current_blink = left_blinking or right_blinking

            # Handle blink state for claw control
            if current_blink and not blink_detected:
                blink_detected = True
                claw_state = not claw_state  # Toggle claw state
                if claw_state:
                    arduino.write(b'OPEN\n')  # Open claw
                else:
                    arduino.write(b'CLOSE\n')  # Close claw
            elif not current_blink:
                blink_detected = False

            # Detect eyebrow movement
            eyebrow_state, baseline_position = calculate_eyebrow_movement(
                [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark],
                LEFT_BROW, RIGHT_BROW, baseline_position, movement_threshold
            )

            # Send commands to Arduino based on eyebrow state
            if eyebrow_state == "Up":
                arduino.write(b"UP\n")  # Send "UP" command
            elif eyebrow_state == "Neutral":
                arduino.write(b"DOWN\n")  # Send "DOWN" command

            # Detect mouth status
            mouth_status = detect_mouth_status(face_landmarks)

            # Send mouth status to Arduino
            if mouth_status == "Mouth Open":
                arduino.write(b'EXTEND\n')  # Send signal to Arduino
            else:
                arduino.write(b'RETRACT\n')  # Send signal to Arduino

            # Draw the text information on the frame
            cv2.putText(frame1, f"Gaze: {gaze_direction}", (30, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 1)
            cv2.putText(frame1, f"Claw: {'Open' if claw_state else 'Closed'}", (30, 90), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 1)
            cv2.putText(frame1, f"Eyebrow: {eyebrow_state}", (30, 130), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 1)
            cv2.putText(frame1, f"Mouth: {mouth_status}", (30, 170), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 1)

    # Stack the two frames vertically after resizing them to match the width
    stacked_frames = np.vstack((frame1, frame2))

    # Show the output
    cv2.imshow("Face Tracking and Control", stacked_frames)

    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the resources
cap1.release()
cap2.release()
cv2.destroyAllWindows()


### FACE GESTURE CONTROLLED ROBOTIC ARM

In [39]:
import cv2
import mediapipe as mp
import numpy as np
import serial
import time

# Initialize serial communication with Arduino
def initialize_serial(port, baud_rate=9600):
    try:
        arduino = serial.Serial(port, baud_rate)
        time.sleep(2)  # Allow time for connection to initialize
        print("Serial connection established.")
        return arduino
    except Exception as e:
        print(f"Error initializing serial: {e}")
        return None

# Initialize Mediapipe FaceMesh
def initialize_face_mesh():
    mp_face_mesh = mp.solutions.face_mesh
    return mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to calculate gaze direction
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to detect blinking
def is_blinking(eye_landmarks, threshold=0.2):
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    blink_ratio = vertical_distance / horizontal_distance
    return blink_ratio < threshold

# Function to detect mouth status
def detect_mouth_status(landmarks):
    upper_lip_y = landmarks.landmark[61].y
    lower_lip_y = landmarks.landmark[17].y
    lip_distance = abs(upper_lip_y - lower_lip_y)
    
    left_eye = landmarks.landmark[33]
    right_eye = landmarks.landmark[263]
    eye_distance = abs(left_eye.x - right_eye.x)
    
    threshold = 0.025 + (eye_distance * 0.1)
    return "Mouth Open" if lip_distance > threshold else "Mouth Closed"

# Function to calculate eyebrow movement
def calculate_eyebrow_movement(landmarks, left_brow_indices, right_brow_indices, baseline, movement_threshold):
    left_brow_y = np.mean([landmarks[i][1] for i in left_brow_indices])
    right_brow_y = np.mean([landmarks[i][1] for i in right_brow_indices])
    avg_brow_y = (left_brow_y + right_brow_y) / 2

    if baseline is None:
        baseline = avg_brow_y
        return "Neutral", baseline

    if avg_brow_y < baseline - movement_threshold:
        return "Eyebrows Raised", baseline
    else:
        baseline = 0.9 * baseline + 0.1 * avg_brow_y
        return "Neutral", baseline

# Function to send commands to Arduino
def send_command_to_arduino(arduino, command):
    if arduino:
        try:
            arduino.write(f"{command}\n".encode())
        except Exception as e:
            print(f"Error sending command: {e}")

# Function to overlay detection results
def overlay_text(frame, text, position, color=(0, 255, 0)):
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)

# Function to process face landmarks
def process_face_landmarks(face_landmarks, arduino, baseline_position, movement_threshold, frame):
    global blink_detected, claw_state
    
    LEFT_BROW = [46, 53, 52, 65, 55, 63, 105, 66, 107]
    RIGHT_BROW = [276, 283, 282, 295, 285, 293, 334, 296, 336]
    
    # Gaze detection
    left_iris = face_landmarks.landmark[468]
    right_iris = face_landmarks.landmark[473]
    left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]
    right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]
    
    avg_gaze_ratio = (calculate_gaze_direction(left_iris, left_eye_corners) +
                      calculate_gaze_direction(right_iris, right_eye_corners)) / 2
    gaze_direction = "Center"
    if avg_gaze_ratio < 0.35:
        send_command_to_arduino(arduino, 'L')
        gaze_direction = "Left"
        gaze_color = (0, 0, 255)  # Red
    elif avg_gaze_ratio > 0.65:
        send_command_to_arduino(arduino, 'R')
        gaze_direction = "Right"
        gaze_color = (0, 0, 255)  # Red
    else:
        gaze_color = (0, 255, 0)  # Green (default)
    overlay_text(frame, f"Gaze: {gaze_direction}", (10, 30), gaze_color)

    # Blink detection and claw state
    left_eye_landmarks = [face_landmarks.landmark[133], face_landmarks.landmark[159], face_landmarks.landmark[145], face_landmarks.landmark[33]]
    right_eye_landmarks = [face_landmarks.landmark[362], face_landmarks.landmark[386], face_landmarks.landmark[374], face_landmarks.landmark[263]]
    current_blink = is_blinking(left_eye_landmarks) or is_blinking(right_eye_landmarks)
    if current_blink and not blink_detected:
        blink_detected = True
        claw_state = not claw_state
        send_command_to_arduino(arduino, 'OPEN' if claw_state else 'CLOSE')
    elif not current_blink:
        blink_detected = False
    claw_color = (0, 255, 0) if claw_state else (0, 0, 255)  # Green if open, Red if closed
    overlay_text(frame, f"Claw: {'Open' if claw_state else 'Closed'}", (10, 60), claw_color)

    # Eyebrow movement
    eyebrow_state, baseline_position = calculate_eyebrow_movement(
        [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark],
        LEFT_BROW, RIGHT_BROW, baseline_position, movement_threshold
    )
    send_command_to_arduino(arduino, "UP" if eyebrow_state == "Eyebrows Raised" else "DOWN")
    eyebrow_color = (0, 0, 255) if eyebrow_state == "Eyebrows Raised" else (0, 255, 0)  # Red if raised, Green if neutral
    overlay_text(frame, f"Eyebrows: {eyebrow_state}", (10, 90), eyebrow_color)

    # Mouth status
    mouth_status = detect_mouth_status(face_landmarks)
    send_command_to_arduino(arduino, 'EXTEND' if mouth_status == "Mouth Open" else 'RETRACT')
    mouth_color = (0, 0, 255) if mouth_status == "Mouth Open" else (0, 255, 0)  # Red if open, Green if closed
    overlay_text(frame, f"Mouth: {mouth_status}", (10, 120), mouth_color)

    return baseline_position

# Main function
def main():
    global blink_detected, claw_state
    blink_detected = False
    claw_state = False
    
    # Initialize resources
    arduino = initialize_serial('COM3')
    face_mesh = initialize_face_mesh()
    cap1 = cv2.VideoCapture(0)
    cap2 = cv2.VideoCapture(1)
    
    baseline_position = None
    movement_threshold = 0.01

    while cap1.isOpened() and cap2.isOpened():
        success1, frame1 = cap1.read()
        success2, frame2 = cap2.read()
        if not success1 or not success2:
            print("Error reading frames.")
            break
        
        # Flip the frames horizontally for better visualization
        frame1 = cv2.flip(frame1, 1)
        frame2 = cv2.flip(frame2, 1)
        
        # Convert frames to RGB for processing
        frame_rgb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
        
        results = face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                baseline_position = process_face_landmarks(face_landmarks, arduino, baseline_position, movement_threshold, frame1)

        # Resize frames for better visualization
        frame1_resized = cv2.resize(frame1, (640, 400))
        frame2_resized = cv2.resize(frame2, (640, 400))

        # Stack frames verticallyq
        stacked_frame = np.vstack((frame1_resized, frame2_resized))

        # Show the stacked frames with overlayed text
        cv2.imshow("Feed", stacked_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Serial connection established.


### WITH LANDMARKS

In [55]:
#### import cv2
import mediapipe as mp
import numpy as np
import serial
import time

# Initialize serial communication with Arduino
def initialize_serial(port, baud_rate=9600):
    try:
        arduino = serial.Serial(port, baud_rate)
        time.sleep(2)  # Allow time for connection to initialize
        print("Serial connection established.")
        return arduino
    except Exception as e:
        print(f"Error initializing serial: {e}")
        return None

# Initialize Mediapipe FaceMesh
def initialize_face_mesh():
    mp_face_mesh = mp.solutions.face_mesh
    return mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to calculate gaze direction
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to detect blinking
def is_blinking(eye_landmarks, threshold=0.2):
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    blink_ratio = vertical_distance / horizontal_distance
    return blink_ratio < threshold

# Function to detect mouth status
def detect_mouth_status(landmarks):
    upper_lip_y = landmarks.landmark[61].y
    lower_lip_y = landmarks.landmark[17].y
    lip_distance = abs(upper_lip_y - lower_lip_y)
    
    left_eye = landmarks.landmark[33]
    right_eye = landmarks.landmark[263]
    eye_distance = abs(left_eye.x - right_eye.x)
    
    threshold = 0.025 + (eye_distance * 0.1)
    return "Mouth Open" if lip_distance > threshold else "Mouth Closed"

# Function to calculate eyebrow movement
def calculate_eyebrow_movement(landmarks, left_brow_indices, right_brow_indices, baseline, movement_threshold):
    left_brow_y = np.mean([landmarks[i][1] for i in left_brow_indices])
    right_brow_y = np.mean([landmarks[i][1] for i in right_brow_indices])
    avg_brow_y = (left_brow_y + right_brow_y) / 2

    if baseline is None:
        baseline = avg_brow_y
        return "Neutral", baseline

    if avg_brow_y < baseline - movement_threshold:
        return "Eyebrows Raised", baseline
    else:
        baseline = 0.9 * baseline + 0.1 * avg_brow_y
        return "Neutral", baseline

# Function to send commands to Arduino
def send_command_to_arduino(arduino, command):
    if arduino:
        try:
            arduino.write(f"{command}\n".encode())
        except Exception as e:
            print(f"Error sending command: {e}")

# Function to overlay detection results
def overlay_text(frame, text, position, color=(0, 255, 0)):
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)

# Function to process face landmarks
def process_face_landmarks(face_landmarks, arduino, baseline_position, movement_threshold, frame):
    global blink_detected, claw_state
    
    LEFT_BROW = [46, 53, 52, 65, 55, 63, 105, 66, 107]
    RIGHT_BROW = [276, 283, 282, 295, 285, 293, 334, 296, 336]
    
    # Gaze detection
    left_iris = face_landmarks.landmark[468]
    right_iris = face_landmarks.landmark[473]
    left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]
    right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]
    
    avg_gaze_ratio = (calculate_gaze_direction(left_iris, left_eye_corners) +
                      calculate_gaze_direction(right_iris, right_eye_corners)) / 2
    gaze_direction = "Center"
    if avg_gaze_ratio < 0.35:
        send_command_to_arduino(arduino, 'L')
        gaze_direction = "Left"
        gaze_color = (0, 0, 255)  # Red
    elif avg_gaze_ratio > 0.65:
        send_command_to_arduino(arduino, 'R')
        gaze_direction = "Right"
        gaze_color = (0, 0, 255)  # Red
    else:
        gaze_color = (0, 255, 0)  # Green (default)
    overlay_text(frame, f"Gaze: {gaze_direction}", (10, 30), gaze_color)

    # Blink detection and claw state
    left_eye_landmarks = [face_landmarks.landmark[133], face_landmarks.landmark[159], face_landmarks.landmark[145], face_landmarks.landmark[33]]
    right_eye_landmarks = [face_landmarks.landmark[362], face_landmarks.landmark[386], face_landmarks.landmark[374], face_landmarks.landmark[263]]
    current_blink = is_blinking(left_eye_landmarks) or is_blinking(right_eye_landmarks)
    if current_blink and not blink_detected:
        blink_detected = True
        claw_state = not claw_state
        send_command_to_arduino(arduino, 'OPEN' if claw_state else 'CLOSE')
    elif not current_blink:
        blink_detected = False
    claw_color = (0, 255, 0) if claw_state else (0, 0, 255)  # Green if open, Red if closed
    overlay_text(frame, f"Claw: {'Open' if claw_state else 'Closed'}", (10, 60), claw_color)

    # Eyebrow movement
    eyebrow_state, baseline_position = calculate_eyebrow_movement(
        [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark],
        LEFT_BROW, RIGHT_BROW, baseline_position, movement_threshold
    )
    send_command_to_arduino(arduino, "UP" if eyebrow_state == "Eyebrows Raised" else "DOWN")
    eyebrow_color = (0, 0, 255) if eyebrow_state == "Eyebrows Raised" else (0, 255, 0)  # Red if raised, Green if neutral
    overlay_text(frame, f"Eyebrows: {eyebrow_state}", (10, 90), eyebrow_color)

    # Mouth status
    mouth_status = detect_mouth_status(face_landmarks)
    send_command_to_arduino(arduino, 'EXTEND' if mouth_status == "Mouth Open" else 'RETRACT')
    mouth_color = (0, 0, 255) if mouth_status == "Mouth Open" else (0, 255, 0)  # Red if open, Green if closed
    overlay_text(frame, f"Mouth: {mouth_status}", (10, 120), mouth_color)

    # Draw the pupil landmarks in red for both eyes
    left_pupil_landmark = face_landmarks.landmark[468]  # Left pupil center
    right_pupil_landmark = face_landmarks.landmark[473]  # Right pupil center
    height, width, _ = frame.shape
    
    left_pupil_x, left_pupil_y = int(left_pupil_landmark.x * width), int(left_pupil_landmark.y * height)
    right_pupil_x, right_pupil_y = int(right_pupil_landmark.x * width), int(right_pupil_landmark.y * height)
    
    # Red dot for both pupils
    cv2.circle(frame, (left_pupil_x, left_pupil_y), 3, (0, 0, 255), -1)
    cv2.circle(frame, (right_pupil_x, right_pupil_y), 3, (0, 0, 255), -1)

    return baseline_position

# Main function
def main():
    global blink_detected, claw_state
    blink_detected = False
    claw_state = False
    
    # Initialize resources
    arduino = initialize_serial('COM3')
    face_mesh = initialize_face_mesh()
    cap1 = cv2.VideoCapture(0)
    cap2 = cv2.VideoCapture(1)
    
    baseline_position = None
    movement_threshold = 0.01

    while cap1.isOpened() and cap2.isOpened():
        success1, frame1 = cap1.read()
        success2, frame2 = cap2.read()
        if not success1 or not success2:
            print("Error reading frames.")
            break
        
        # Flip the frames horizontally for better visualization
        frame1 = cv2.flip(frame1, 1)
        frame2 = cv2.flip(frame2, 1)
        
        # Convert frames to RGB for processing
        frame_rgb = cv2.cvtColor(frame1, cv2.COLOR_BGR2RGB)
        
        results = face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                baseline_position = process_face_landmarks(face_landmarks, arduino, baseline_position, movement_threshold, frame1)

        # Resize frames for better visualization
        frame1_resized = cv2.resize(frame1, (640, 420))
        frame2_resized = cv2.resize(frame2, (640, 420))

        # Stack frames vertically
        stacked_frame = np.vstack((frame1_resized, frame2_resized))

        # Show the stacked frames with overlayed text
        cv2.imshow("Feed", stacked_frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap1.release()
    cap2.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Serial connection established.


### MEASURING RESPONSE TIME

In [ ]:
import cv2
import mediapipe as mp
import time
import numpy as np

# Initialize Mediapipe FaceMesh
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Webcam capture
cap = cv2.VideoCapture(0)

# Variables for gesture state
blink_detected = False  # Tracks if blink was detected in the previous frame
eyebrow_state = "Neutral"

# Function to calculate the gaze direction
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to calculate blinking
def is_blinking(eye_landmarks, threshold=0.2):
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    blink_ratio = vertical_distance / horizontal_distance
    return blink_ratio < threshold

# Function to calculate the eyebrow movement (detect upward movement only)
def calculate_eyebrow_movement(landmarks, left_brow_indices, right_brow_indices, baseline, movement_threshold):
    left_brow_y = np.mean([landmarks[i][1] for i in left_brow_indices])
    right_brow_y = np.mean([landmarks[i][1] for i in right_brow_indices])
    avg_brow_y = (left_brow_y + right_brow_y) / 2

    if baseline is None:
        baseline = avg_brow_y
        return "Neutral", baseline

    if avg_brow_y < baseline - movement_threshold:
        return "Up", baseline
    else:
        baseline = 0.9 * baseline + 0.1 * avg_brow_y
        return "Neutral", baseline

# Function to process frames and measure gesture recognition time
def process_gestures():
    cap = cv2.VideoCapture(0)

    # Initialize baseline and movement threshold for eyebrow detection
    baseline_position = None
    movement_threshold = 0.01

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            continue

        frame = cv2.flip(frame, 1)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        start_time = time.time()  # Start the timer when the frame is being processed
        results = face_mesh.process(frame_rgb)
        
        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                current_blink = False
                eyebrow_state, baseline_position = calculate_eyebrow_movement(
                    [(lm.x, lm.y, lm.z) for lm in face_landmarks.landmark],
                    [46, 53, 52, 65, 55, 63, 105, 66, 107],  # Left eyebrow
                    [276, 283, 282, 295, 285, 293, 334, 296, 336],  # Right eyebrow
                    baseline_position,
                    movement_threshold
                )
                
                # Detect eye blinking
                left_eye_landmarks = [
                    face_landmarks.landmark[133],  # Left corner
                    face_landmarks.landmark[159],  # Top
                    face_landmarks.landmark[145],  # Bottom
                    face_landmarks.landmark[33],   # Right corner
                ]
                right_eye_landmarks = [
                    face_landmarks.landmark[362],  # Left corner
                    face_landmarks.landmark[386],  # Top
                    face_landmarks.landmark[374],  # Bottom
                    face_landmarks.landmark[263],  # Right corner
                ]
                left_blinking = is_blinking(left_eye_landmarks)
                right_blinking = is_blinking(right_eye_landmarks)
                current_blink = left_blinking or right_blinking
                
                # Detect eye gaze direction
                left_iris = face_landmarks.landmark[468]
                right_iris = face_landmarks.landmark[473]
                left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]
                right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]

                left_gaze_ratio = calculate_gaze_direction(left_iris, left_eye_corners)
                right_gaze_ratio = calculate_gaze_direction(right_iris, right_eye_corners)
                avg_gaze_ratio = (left_gaze_ratio + right_gaze_ratio) / 2

                # Print out the gesture recognition results
                if current_blink:
                    print("Blink detected!")
                if eyebrow_state == "Up":
                    print("Eyebrow raised!")
                if avg_gaze_ratio < 0.35:
                    print("Looking left!")
                elif avg_gaze_ratio > 0.65:
                    print("Looking right!")
                
                # Calculate time taken for gesture detection and print the result
                end_time = time.time()  # Capture the time after processing the gesture
                gesture_recognition_time = end_time - start_time
                print(f"Gesture Recognition Time: {gesture_recognition_time:.4f} seconds")

                # Display on the frame
                cv2.putText(frame, f"Gaze: {'Left' if avg_gaze_ratio < 0.35 else 'Right' if avg_gaze_ratio > 0.65 else 'Center'}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(frame, f"Blink: {'Detected' if current_blink else 'Not Detected'}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
                cv2.putText(frame, f"Eyebrow: {eyebrow_state}", (10, 90), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Show the output frame with gestures detected
        cv2.imshow("Gesture Recognition", frame)

        # Exit on pressing 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Run the gesture processing function
process_gestures()


In [24]:
import cv2
import mediapipe as mp
import numpy as np
import time

# Initialize Mediapipe FaceMesh
def initialize_face_mesh():
    mp_face_mesh = mp.solutions.face_mesh
    return mp_face_mesh.FaceMesh(refine_landmarks=True, min_detection_confidence=0.5, min_tracking_confidence=0.5)

# Function to calculate gaze direction
def calculate_gaze_direction(iris_center, eye_corners):
    left_corner = eye_corners[0]
    right_corner = eye_corners[1]
    horizontal_ratio = (iris_center.x - left_corner.x) / (right_corner.x - left_corner.x)
    return horizontal_ratio

# Function to detect blinking
def is_blinking(eye_landmarks, threshold=0.2):
    top = np.array([eye_landmarks[1].x, eye_landmarks[1].y])
    bottom = np.array([eye_landmarks[2].x, eye_landmarks[2].y])
    left = np.array([eye_landmarks[0].x, eye_landmarks[0].y])
    right = np.array([eye_landmarks[3].x, eye_landmarks[3].y])

    vertical_distance = np.linalg.norm(top - bottom)
    horizontal_distance = np.linalg.norm(left - right)

    blink_ratio = vertical_distance / horizontal_distance
    return blink_ratio < threshold

# Function to overlay detection results
def overlay_text(frame, text, position, color=(0, 255, 0)):
    cv2.putText(frame, text, position, cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)

# Function to process face landmarks
def process_face_landmarks(face_landmarks, baseline_position, movement_threshold, frame, eye_closed_time):
    global blink_detected, claw_state
    
    LEFT_BROW = [46, 53, 52, 65, 55, 63, 105, 66, 107]
    RIGHT_BROW = [276, 283, 282, 295, 285, 293, 334, 296, 336]
    
    # Gaze detection
    left_iris = face_landmarks.landmark[468]
    right_iris = face_landmarks.landmark[473]
    left_eye_corners = [face_landmarks.landmark[33], face_landmarks.landmark[133]]
    right_eye_corners = [face_landmarks.landmark[362], face_landmarks.landmark[263]]
    
    avg_gaze_ratio = (calculate_gaze_direction(left_iris, left_eye_corners) +
                      calculate_gaze_direction(right_iris, right_eye_corners)) / 2
    gaze_direction = "Center"
    if avg_gaze_ratio < 0.35:
        gaze_direction = "Left"
        gaze_color = (0, 0, 255)  # Red
    elif avg_gaze_ratio > 0.65:
        gaze_direction = "Right"
        gaze_color = (0, 0, 255)  # Red
    else:
        gaze_color = (0, 255, 0)  # Green (default)
    overlay_text(frame, f"Gaze: {gaze_direction}", (10, 30), gaze_color)

    # Blink detection
    left_eye_landmarks = [face_landmarks.landmark[133], face_landmarks.landmark[159], face_landmarks.landmark[145], face_landmarks.landmark[33]]
    right_eye_landmarks = [face_landmarks.landmark[362], face_landmarks.landmark[386], face_landmarks.landmark[374], face_landmarks.landmark[263]]
    current_blink = is_blinking(left_eye_landmarks) or is_blinking(right_eye_landmarks)
    if current_blink and not blink_detected:
        blink_detected = True
    elif not current_blink:
        blink_detected = False

    # Track how long the eyes are closed for
    if current_blink:
        eye_closed_time += 1
    else:
        eye_closed_time = 0  # Reset the timer if eyes are open

    # Stop the program if eyes are closed for more than a certain threshold (3 seconds or 90 frames)
    if eye_closed_time > 90:  # 3 seconds at 30 FPS = 90 frames
        print("Eyes closed for too long, exiting the program...")
        # Reset arm to initial position (send reset commands here)
        send_reset_command()
        cap.release()
        cv2.destroyAllWindows()
        exit()

    return baseline_position, eye_closed_time

# Function to send reset command to robot (example - you can adjust the actual control logic)
def send_reset_command():
    print("Resetting robot to initial position...")
    # Reset joint positions to their initial values
    # Example: Reset the base, claw, shoulder, and elbow positions here.
    # You could send reset commands or set variables to initial positions.
    pass

# Main function
def main():
    global blink_detected, claw_state, eye_closed_time
    blink_detected = False
    claw_state = False
    eye_closed_time = 0  # Timer for eye closure detection
    
    # Initialize resources
    face_mesh = initialize_face_mesh()
    cap = cv2.VideoCapture(0)
    
    baseline_position = None
    movement_threshold = 0.01

    while cap.isOpened():
        success, frame = cap.read()
        if not success:
            print("Error reading frames.")
            break
        
        # Flip the frame horizontally for better visualization
        frame = cv2.flip(frame, 1)
        
        # Convert frames to RGB for processing
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        results = face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                baseline_position, eye_closed_time = process_face_landmarks(face_landmarks, baseline_position, movement_threshold, frame, eye_closed_time)

        # Resize frames for better visualization
        frame_resized = cv2.resize(frame, (640, 420))

        # Show the frame with overlayed text
        cv2.imshow("Feed", frame_resized)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()
